# Electricity prediction

Source: https://medium.com/@kylejones_47003/neural-networks-for-time-series-with-tensorflow-keras-in-python-0479918268f1

In [7]:
import pandas as pd, numpy as np, os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # settings like template by default

from dotenv import load_dotenv
from pathlib import Path

load_dotenv(dotenv_path = Path('.') / '.config')

pio.templates.default = os.getenv("template_plotly") # default template

only_cpu = True
if only_cpu: 
    os.environ["CUDA_VISIBLE_DEVICES"] = str(os.getenv("CUDA_VISIBLE_DEVICES"))
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = str(os.getenv("TF_CPP_MIN_LOG_LEVEL"))  # 0 = all logs, 1 = filter INFO, 2 = filter WARNING, 3 = filter ERROR

import tensorflow as tf
# gpus = tf.config.list_physical_devices('GPU')  # obtiene las gpus instaladas
# if len(gpus) > 0:   # If GPU available
#     tf.config.experimental.set_memory_growth(gpus[0], True)
#     print(gpus)

In [8]:
df = pd.read_csv("data/energy.csv", usecols=["timestamp", "load"])
df.timestamp = pd.to_datetime(df.timestamp)
df.sort_values(by=['timestamp'], inplace=True, ascending=True)
df.set_index('timestamp', inplace=True)
print(f"Data {df.shape}")
df.head(2)

Data (26304, 1)


,load
timestamp,
2012-01-01 00:00:00,2698.0
2012-01-01 01:00:00,2558.0


In [33]:
# Create lagged features
def create_features(data, lag=3):
    X, y = [], []
    for i in range(len(data) - lag):
        X.append(data[i:i + lag])
        y.append(data[i + lag])
    X = np.array(X)
    X = X.reshape((X.shape[0], lag))
    y = np.array(y)
    return X, y

lag = 3
X, y = create_features(df.values, lag=lag)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
print(f"Train labels shape: {y_train.shape}, Test labels shape: {y_test.shape}")

# Normalize the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_train = scaler_X.fit_transform(X_train)
# X_test = scaler.transform(X_test)
y_train = scaler_y.fit_transform(y_train)

# Build a simple feedforward neural network
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(lag,)),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(7, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=False, validation_split=0.1)

Train shape: (21040, 3), Test shape: (5261, 3)
Train labels shape: (21040, 1), Test labels shape: (5261, 1)


In [27]:
X_test_scaled = scaler_X.transform(X_test)
y_pred_scaled = model.predict(X_test_scaled, verbose=False)
# Inverse transform the prediction
y_pred = scaler_y.inverse_transform(y_pred_scaled)

rmse = np.sqrt(np.mean((y_test - y_pred)**2))
print("RMSE:", rmse)

RMSE: 62.2721949706431


In [32]:
df_test = df.tail(X_test.shape[0]).copy()
df_test['y_pred'] = y_pred.flatten()
df_test = df_test.head(24)
print(df_test)
fig = px.line(df_test, x = df_test.index, y= ['load', 'y_pred'],
            labels={'value': 'Energy', 'variable': 'Values'},
            title=f'Feedforward Neural Network Forecast RMSE: {rmse:.3f}')
fig.show()

                       load       y_pred
timestamp                               
2014-05-27 00:00:00  2343.0  2453.846436
2014-05-27 01:00:00  2259.0  2339.615234
2014-05-27 02:00:00  2228.0  3397.627441
2014-05-27 03:00:00  2254.0  3644.910889
2014-05-27 04:00:00  2349.0  3545.921631
2014-05-27 05:00:00  2582.0  3612.176025
2014-05-27 06:00:00  3026.0  3209.930908
2014-05-27 07:00:00  3311.0  2262.046875
2014-05-27 08:00:00  3371.0  3871.259277
2014-05-27 09:00:00  3446.0  4021.327637
2014-05-27 10:00:00  3476.0  2340.603027
2014-05-27 11:00:00  3490.0  3331.357910
2014-05-27 12:00:00  3464.0  2606.026367
2014-05-27 13:00:00  3461.0  3890.755615
2014-05-27 14:00:00  3425.0  3355.646729
2014-05-27 15:00:00  3406.0  3703.383545
2014-05-27 16:00:00  3436.0  2435.313965
2014-05-27 17:00:00  3485.0  3172.950684
2014-05-27 18:00:00  3470.0  3376.388428
2014-05-27 19:00:00  3447.0  3655.582031
2014-05-27 20:00:00  3387.0  3122.312500
2014-05-27 21:00:00  3133.0  2726.795898
2014-05-27 22:00